In [1]:
import sys
sys.path.append('/workspace/transformers')
print(sys.path)

['/workspace/representation-engineering/examples/honesty', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/workspace/transformers/src', '/workspace/representation-engineering', '/workspace/repeng', '/usr/lib/python3/dist-packages', '/workspace/transformers']


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from transformers import AutoTokenizer, AutoConfig, pipeline, AutoModelForCausalLM
from transformers.generation.streamers import TextStreamer
import torch
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from functools import partial

from repe.rep_control_contrast_vec import ContrastVecLlamaForCausalLM, ContrastVecMistralForCausalLM

In [4]:
from typing import *
from queue import Queue
class Streamer(TextStreamer):
    def __init__(
        self, tokenizer: "AutoTokenizer", total=128, skip_prompt: bool = False, timeout: Optional[float] = None, **decode_kwargs
    ):
        super().__init__(tokenizer, skip_prompt, **decode_kwargs)
        self.text_queue = Queue()
        self.stop_signal = None
        self.timeout = timeout
        self.repe = None
        self.c = 0
        self.total = 128

    def on_finalized_text(self, text: str, stream_end: bool = False, corrs=None):
        """Put the new text in the queue. If the stream is ending, also put a stop signal in the queue."""
        self.text_queue.put((text,corrs), timeout=self.timeout)
        if stream_end:
            self.text_queue.put((self.stop_signal,None), timeout=self.timeout)
        new_repe = {
            "dynamic_compute": False,
            "control_layers": list(range(18, 25)),
            "repe_vecs": get_random_repe_vecs(self.c, self.total),
            "compute_corr": None
        }
        self.set_repe(new_repe)
        super().on_finalized_text(text)
        self.c += 1
        
    def set_repe(self, repe):
        self.repe = repe
    
    def get_repe(self):
        return self.repe

    def __iter__(self):
        return self

    # returned is gonna be Tuple (tok, corr)
    def __next__(self):
        value = self.text_queue.get(timeout=self.timeout)
        if value == self.stop_signal:
            raise StopIteration()
        else:
            return value


In [5]:
from transformers import LlamaForCausalLM

name = "meta-llama/Meta-Llama-3-8b-Instruct"
model = LlamaForCausalLM.from_pretrained(
    name, 
    torch_dtype=torch.bfloat16,
    device_map='cuda'
)

tokenizer = AutoTokenizer.from_pretrained(name)

tokenizer.padding_side = 'left'
tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
tokenizer.pad_token = tokenizer.eos_token

model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [21]:
temple = '''<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Answer the user's question in as much detail as possible. Be witty and precise but don't answer incorrectly!<|eot_id|><|start_header_id|>user<|end_header_id|>{prompt}<|start_header_id|>assistant<|end_header_id|>'''

In [29]:
from transformers import pipeline
import random

NUM_TOKEN = 256  # Example number, set as required
happy_vec = torch.load('../data/disgust_rep_reader.pt')

for k,v in happy_vec.directions.items():
    happy_vec.directions[k] = torch.tensor(v).cuda()
    s = happy_vec.direction_signs[k]
    happy_vec.direction_signs[k] = torch.tensor(s).cuda()

print(happy_vec)
def get_random_repe_vecs(c, total): 
   # print(f'({c/total})', end='')
    return {"happiness": (happy_vec, 2)}

streamer = Streamer(tokenizer, total=NUM_TOKEN)

pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=NUM_TOKEN,
    streamer=streamer,
    do_sample=True,
    use_cache=False
)
prompt = "You are a conversational agent who has a conversation with the user. USER: What is your immediate reaction when you find hair in soup?"
pipe(temple.format(prompt=prompt))

begin pipeline generation...
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Answer the user's question in as much detail as possible. Be witty and precise but don't answer incorrectly!<|eot_id|><|start_header_id|>user<|end_header_id|>You are a conversational agent who has a conversation with the user. USER: What is your immediate reaction when you find hair in soup?<|start_header_id|>assistant<|end_header_id|>

The age-old question that gets to the root of our deepest fears (pun intended)! As a conversational agent, I must admit that I'm a bit of a soup connoisseur, and the thought of finding hair in my soup is, quite frankly, a culinary horror show!

My immediate reaction would be a mix of shock, disgust, and a dash of humor (because, let's face it, it's a pretty ridiculous situation). I'd likely do a double-take, wondering if I'm hallucinating or if my taste buds are playing a cruel joke on me. I might even do a quick scan of the surrounding area to make sure I'm not in

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer the user\'s question in as much detail as possible. Be witty and precise but don\'t answer incorrectly!<|eot_id|><|start_header_id|>user<|end_header_id|>You are a conversational agent who has a conversation with the user. USER: What is your immediate reaction when you find hair in soup?<|start_header_id|>assistant<|end_header_id|>\n\nThe age-old question that gets to the root of our deepest fears (pun intended)! As a conversational agent, I must admit that I\'m a bit of a soup connoisseur, and the thought of finding hair in my soup is, quite frankly, a culinary horror show!\n\nMy immediate reaction would be a mix of shock, disgust, and a dash of humor (because, let\'s face it, it\'s a pretty ridiculous situation). I\'d likely do a double-take, wondering if I\'m hallucinating or if my taste buds are playing a cruel joke on me. I might even do a quick scan of the surrounding area to make sure I\'m 